# Matches analysis


In [1]:
import math
import os
import pandas as pd
import re
import ast
import xarray
folder_list= ["COMA_ground_associated","Word2Vec_ground_associated"]
GT = {"FHIR_C-CDA":pd.read_csv("../mappings/FHIR_to_C-CDA_._delimiter.csv"), "ODATIS_AERIS":pd.read_csv("../mappings/ODATIS_to_AERIS_._delimiter_only_leaf.csv.csv"), "AERIS_FHIR":pd.read_csv("../mappings/AERIS_to_FHIR_._delimiter.csv")}
model = {"C-CDA":pd.read_csv("../models/C-CDA_model.csv"),"FHIR":pd.read_csv("../models/FHIR_model.csv"),"ODATIS":pd.read_csv("../models/ODATIS_model.csv"),"AERIS":pd.read_csv("../models/AERIS_model.csv")}

list_df =[]
expe_list = []
for folder in folder_list:
    file_list = list(os.walk(folder))[0][2]
    file_list.sort()
    for file_name in file_list:

        expe = file_name.split("_")[0]
        model_A_with_size, model_B_with_size = re.findall("[^_]*_[0-9]{,2}-sizedpath|[^_]*_[0-9]{,2}-sized_path", file_name)

        res_with_GT_df = pd.read_csv(folder+"/"+file_name)
        ground_truth_df = GT[model_A_with_size.split("_")[0]+"_"+model_B_with_size.split("_")[0]]

        model_A_df = model[model_A_with_size.split("_")[0]]
        model_B_df = model[model_B_with_size.split("_")[0]]
        key_list = []
        key_from_model_list = []
        is_key_unique = []
        match_done_list = []
        dist_list = []
        match_that_should_be_done_list = []
        is_match_unique = []

        potential_good_match_list = []
        for row in res_with_GT_df.iloc():
            potential_good_match = pd.Series(ast.literal_eval(row["match_that_should_be_done"]),dtype=str)

            potential_good_match = potential_good_match[potential_good_match.str.endswith(row["match_done"])]

            potential_good_match_list.append(list(potential_good_match))
            key,match_done,dist,match_that_should_be_done=list(row.keys())
            key_list.append(row[key])
            key_from_model_list.append(list(model_A_df[model_A_df["Unnamed: 0"].str.endswith(row[key])]["Unnamed: 0"]))
            match_done_list.append(row[match_done])
            dist_list.append(row[dist])
            match_that_should_be_done_list.append(row[match_that_should_be_done])
            if len(list(model_A_df[model_A_df["Unnamed: 0"].str.endswith(row[key])]["Unnamed: 0"]))==1:
                is_key_unique.append(True)
            else:
                is_key_unique.append(False)
            if len(potential_good_match)==1:
                is_match_unique.append(True)
            else:
                is_match_unique.append(False)

        expe_list.append(expe+"_"+model_A_with_size+"_"+model_B_with_size)

        list_df.append(pd.DataFrame(data={key:key_list, "key_from_model":key_from_model_list, "is_key_unique":is_key_unique, match_done:match_done_list, dist:dist_list,match_that_should_be_done:match_that_should_be_done_list, "is_match_unique": is_match_unique,"potential_good_match":potential_good_match_list   } ))

    # xarray.concat([df.to_xarray() for df in list_df], dim="expe")

res = pd.concat(list_df,keys=expe_list, names=["Experiment", None])
res.to_csv("all_expe_results.csv")

In [10]:
# GT = {"FHIR_C-CDA":pd.read_csv("../mappings/FHIR_to_C-CDA_._delimiter.csv"), "ODATIS_AERIS":pd.read_csv("../mappings/ODATIS_to_AERIS_._delimiter_only_leaf.csv.csv")}

##############################



list_of_expe = list(set(res.index.get_level_values(0)))

precis_list_with_ambiguity = []
recall_list_with_ambiguity  = []
f1_list_with_ambiguity  = []
precis_list_witout_abiguity = []
recall_list_without_ambiguity  = []
f1_list_without_ambiguity  = []
error_list = []

computed_evaluation = []
for expe in list_of_expe:
    try :
        result_expe = res.loc[expe]

        print(expe)

        model_A_with_size, model_B_with_size = re.findall("[^_]*_[0-9]{,2}-sizedpath|[^_]*_[0-9]{,2}-sized_path", expe)
        ground_truth_df = GT[model_A_with_size.split("_")[0]+"_"+model_B_with_size.split("_")[0]]
        match_in_gt = len(ground_truth_df)

        expe_res = res.loc[expe]

        good_match = 0
        good_match_without_ambi_on_key = 0
        good_match_without_ambi_on_match = 0
        good_match_without_ambi = 0
        total_match_done = len(expe_res)

        for row in result_expe.iloc:
            if len(row["potential_good_match"]) > 0:
                good_match+=1
                if row["is_key_unique"]:
                    good_match_without_ambi_on_key +=1
                if row["is_match_unique"]:
                    good_match_without_ambi_on_match+=1
                if row["is_match_unique"] and row["is_key_unique"]:
                    good_match_without_ambi += 1

        precision = (good_match/total_match_done)
        recall = (good_match/match_in_gt)
        if precision != 0 and recall!=0:
            f1_score = 2*((precision*recall)/(precision+recall))
        else :
            f1_score = 0

        computed_evaluation.append((expe,expe.split("_")[0],model_A_with_size.split("_")[0], model_B_with_size.split("_")[0],int(model_A_with_size.split("_")[1].split("-")[0]),int(model_B_with_size.split("_")[1].split("-")[0]),precision,recall,f1_score,match_in_gt,total_match_done,good_match,good_match_without_ambi_on_key,good_match_without_ambi_on_match,good_match_without_ambi))
    except Exception as e :
        error_list.append(expe)

columns = ("experiment","tool","model_1","model_2","model_1_path_size","model_2_path_size","precision", "recall", "F1-score","total_matches_in_ground_truth","total_matches","good_match","good_match_without_ambi_on_key","good_match_without_ambi_on_match","good_match_without_ambi")
computed_evaluation_df = pd.DataFrame(data=computed_evaluation, columns=columns)
computed_evaluation_df.to_csv("validation_metrics_computed.csv")

multi_ODATIS_4-sizedpath_AERIS_2-sized_path
multi_FHIR_5-sizedpath_C-CDA_9-sized_path
COMA_FHIR_2-sizedpath_C-CDA_14-sized_path
euclidian_FHIR_2-sizedpath_C-CDA_18-sized_path
COMA_FHIR_5-sizedpath_C-CDA_13-sized_path
COMA_FHIR_1-sizedpath_C-CDA_12-sized_path
euclidian_ODATIS_9-sizedpath_AERIS_2-sized_path
multi_ODATIS_2-sizedpath_AERIS_3-sized_path
euclidian_ODATIS_11-sizedpath_AERIS_0-sized_path
multi_FHIR_3-sizedpath_C-CDA_2-sized_path
multi_FHIR_0-sizedpath_C-CDA_2-sized_path
multi_FHIR_0-sizedpath_C-CDA_19-sized_path
multi_FHIR_2-sizedpath_C-CDA_0-sized_path
COMA_FHIR_6-sizedpath_C-CDA_1-sized_path
euclidian_FHIR_2-sizedpath_C-CDA_16-sized_path
multi_FHIR_1-sizedpath_C-CDA_17-sized_path
COMA_FHIR_2-sizedpath_C-CDA_17-sized_path
euclidian_FHIR_2-sizedpath_C-CDA_1-sized_path
multi_FHIR_1-sizedpath_C-CDA_16-sized_path
multi_ODATIS_6-sizedpath_AERIS_2-sized_path
multi_FHIR_3-sizedpath_C-CDA_5-sized_path
euclidian_FHIR_0-sizedpath_C-CDA_18-sized_path
euclidian_ODATIS_4-sizedpath_AERIS_0

In [18]:






def select_token(key_list, number, delimiter, split_fun):
    return list(map(lambda x: delimiter.join(split_fun(x)[-number:]), key_list))



def get_max_path_length(model_df, delimiter):
    """
    Retourne la taille des chemins et le nombre d'occurence de cette taille dans le model
    :param model_df: list : list of path
    :return: int : max path size / number of node in the longest path in the model
    """
    aux_counter = 0
    for key_path in model_df:
        if len(key_path.split(delimiter)) > aux_counter:
            aux_counter = len(key_path.split(delimiter))

    return aux_counter

res_csv = pd.read_csv("validation_metrics_computed.csv")
res_csv = res_csv.assign(model_1_size=None,model_2_size=None)
model_list = ["ODATIS","FHIR","AERIS"]
model_len = {}
delimiter="."
for model_name in model_list :
    modelA=pd.read_csv("../models/"+model_name+"_model.csv").set_index("Unnamed: 0").transpose()

    for sub_path_A_size in range(get_max_path_length(modelA.columns, delimiter)):

        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_size" ]=int(len(pd.DataFrame(columns=(list(set(select_token(modelA.columns, sub_path_A_size + 1, delimiter,
                                                          lambda x: x.split(delimiter)))))).columns))

model_list = ["AERIS","C-CDA","FHIR"]
model_len = {}
for model_name in model_list :
    modelA=pd.read_csv("../models/"+model_name+"_model.csv").set_index("Unnamed: 0").transpose()
    print(get_max_path_length(modelA.columns, delimiter))
    for sub_path_A_size in range(get_max_path_length(modelA.columns, delimiter)):

        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_size" ]=int(len(pd.DataFrame(columns=(list(set(select_token(modelA.columns, sub_path_A_size + 1, delimiter,
                                                          lambda x: x.split(delimiter)))))).columns))
res_csv.to_csv("validation_metrics_computed_with_size.csv")

4
20
7


In [19]:
import pandas as pd
import os
from nltk import download
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import wordninja
import re
import collections

download('stopwords')  # Download stopwords list.
path_to_model = "../models/"
stopwords = stopwords.words("english")

def select_token(key_list, number, delimiter, split_fun):
    return list(map(lambda x: delimiter.join(split_fun(x)[-number:]), key_list))



def get_max_path_length(model_df, delimiter):
    """
    Retourne la taille des chemins et le nombre d'occurence de cette taille dans le model
    :param model_df: list : list of path
    :return: int : max path size / number of node in the longest path in the model
    """
    aux_counter = 0
    for key_path in model_df:
        if len(key_path.split(delimiter)) > aux_counter:
            aux_counter = len(key_path.split(delimiter))
    return aux_counter

def get_voc_of_model(model_df, stopwords, remove_stop_word=True, lemmatized_word=True):
    '''
    :param model_df: dataframe contenant le model
    :param stopwords: nltk.corpus contenant les stopwords à enlever
    :param remove_stop_word: Bool : permet de choisir d'enlever les stop words (True) ou de les laisser (False)
    :param lemmatized_word: Bool : permet de choisir de prendre la forme lemmatizé du vocabulaire (True) ou de les laisser tels quels (False)
    :return: Liste des mots du vocabulaire
    '''
    voc= []
    if lemmatized_word:
        lematizer = WordNetLemmatizer()
    for key_path in model_df["Unnamed: 0"]:
        for label in key_path.split("."):
            camel_case_split_of_label = [match.group(0) for match in re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', label)]
            for splited_label in camel_case_split_of_label:
                for word in wordninja.split(splited_label):
                    if lemmatized_word:
                        word_lemmatized = lematizer.lemmatize(word)
                        if remove_stop_word:
                            if word not in stopwords:
                                voc.append(word_lemmatized)
                        else:
                            voc.append(word_lemmatized)
                    else:
                        if remove_stop_word:
                            if word not in stopwords:
                                voc.append(word)
                        else:
                            voc.append(word)
    return voc

def voc_complexity(voc, word_frequency_doc, num_occur):
    '''
    :param voc: vocabulaire (liste des mots distincts)
    :param word_frequency_doc: dataframe contennant la liste des mots de la langue anglaise (récupérés depuis wikipédia) et leur nombre d'occurange
    :param num_occur: Somme des occurance de mots pour le nombre total de mots
    :return: res : dictionnaire contenant les mots du vocabulaires associés à leur fréquence
    + le clé "Not_a_word" qui contient le nombre total de mots du vocabulaire n'étant pas un mot de la langue anglaise
    (peut être : une erreur de récupération des mots / splits des mots dans les labels - un acronyme - un mot qui n'est pas un mot)
    '''
    res = {"Not_a_word":[]}
    for word in voc:
        if word == "null":
            print(word)
        freq = word_frequency_doc[word_frequency[0] == word.lower()][1]
        if freq.size ==1:

           res[word]=freq.item()/num_occur
        else:
            res["Not_a_word"].append(word)
            # res["Not_a_word"]+=1
    return res

def get_average_frequency(voc_frequency):
    '''
    Retourn la moyenne des fréquences de mots du vocabulaire
    :param voc_frequency: dict : les mots et leur fréquence associée
    :return: float
    '''
    sum = 0
    for key in voc_frequency:
        if key !="Not_a_word":
            sum += voc_frequency[key]
    return sum/len(voc_frequency.keys())

def get_weithed_mean_frequency(voc_frequency, all_words):
    '''
    Retourne la moyenne pondérées (par les occurences des mots) des fréquences de mots du vocabulaire
    :param voc_frequency: dict : vocabulaire distinct et fréquence des mots associés
    :param all_words: list : liste totale des mots
    :return: float
    '''
    sum = 0
    word_occur={}
    sum_of_occur = 0
    for word in all_words:
        if word in word_occur:
            word_occur[word]+=1
        else:
            word_occur[word]=1
        sum_of_occur+=1
    for key in voc_frequency:
        if key !="Not_a_word":
            sum += voc_frequency[key]*word_occur[key]
    return sum/sum_of_occur

def get_path_length(model_df):
    '''
    Retourne la taille des chemins et le nombre d'occurence de cette taille dans le model
    :param model_df: Dataframe : contient le modèle
    :return: dict : dictionnaire contenant en clé la taille et en valeur le nombre d'occurence de cette taille dans les chemins du modèle
    '''
    res={}
    for key_path in model_df["Unnamed: 0"]:
        len_key_path=len(key_path.split("."))
        if len_key_path in res:
            res[len_key_path]+=1
        else:
            res[len_key_path]=1
    res = dict(sorted(res.items()))
    return res

def get_node_number(model):
    '''
    A REVOIR : retourne la taille du nombre de noeuds, à revoir car ne marche pas correctement
    :param model:
    :return:
    '''
    nodes=[]

    for path in model["Unnamed: 0"]:
        splitted = path.split(".")
        for index,node in enumerate(splitted):
            nodes.append(".".join(splitted[:index+1]))

    nodes = set(nodes)

    return len(nodes)

def get_child_distrib(model_df):

    for path in model_df["Unnamed: 0"]:
        for index in range(len(path.split(""))):

            pass


res = {}
word_frequency=pd.read_csv("../metric/enwiki-2022-08-29.txt",delimiter=" ", header=None)
word_occur=word_frequency[1].sum()


res_csv = pd.read_csv("validation_metrics_computed_with_size.csv")
res_csv = res_csv.assign(model_1_voc_length=None,model_1_max_path_length=None, model_1_min_path_length=None, model_1_intermediate_node=None, model_1_node_number=None, model_1_average_word_frequency=None, model_1_weighted_average_word_frequency=None,model_2_voc_length=None,model_2_max_path_length=None, model_2_min_path_length=None, model_2_intermediate_node=None, model_2_node_number=None, model_2_average_word_frequency=None, model_2_weighted_average_word_frequency=None)
model_list = ["ODATIS","FHIR","AERIS"]
model_len = {}
delimiter="."
for model_name in model_list :
    modelA_base=pd.read_csv("../models/"+model_name+"_model.csv")
    for sub_path_A_size in range(get_max_path_length(modelA_base.set_index("Unnamed: 0").transpose().columns, delimiter)):

        modelA= pd.DataFrame((list(set(select_token(modelA_base["Unnamed: 0"],  sub_path_A_size + 1, delimiter,
                                                          lambda x: x.split(delimiter))))), columns=["Unnamed: 0"])

        model_voc_occur = get_voc_of_model(modelA, stopwords=stopwords, lemmatized_word=False)
        model_voc = set(model_voc_occur)
        lengths = get_path_length(modelA)
        length_key_list = list(lengths.keys())
        len_length_key_list = len(length_key_list)

        voc_complexity_var = voc_complexity(model_voc,word_frequency,word_occur)

        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_node_number" ]= get_node_number(modelA)

        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_voc_length" ]=len(model_voc)
        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_max_path_length" ]= length_key_list[len_length_key_list-1]
        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_min_path_length" ]= length_key_list[0]
        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_intermediate_node" ]= get_node_number(modelA) - len(modelA)
        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_average_word_frequency" ]= get_average_frequency(voc_complexity_var)
        res_csv.loc[ res_csv.query('model_1=="' + model_name + '" & model_1_path_size=='+str(sub_path_A_size) ).index, "model_1_weighted_average_word_frequency" ]= get_weithed_mean_frequency(voc_complexity_var, model_voc_occur)

model_list = ["AERIS","C-CDA", "FHIR"]
model_len = {}
for model_name in model_list :
    modelA_base=pd.read_csv("../models/"+model_name+"_model.csv")

    for sub_path_A_size in range(get_max_path_length(modelA_base.set_index("Unnamed: 0").transpose().columns, delimiter)):

        modelA= pd.DataFrame((list(set(select_token(modelA_base["Unnamed: 0"],  sub_path_A_size + 1, delimiter,
                                                          lambda x: x.split(delimiter))))), columns=["Unnamed: 0"])

        model_voc_occur = get_voc_of_model(modelA, stopwords=stopwords, lemmatized_word=False)
        model_voc = set(model_voc_occur)

        lengths = get_path_length(modelA)

        length_key_list = list(lengths.keys())
        len_length_key_list = len(length_key_list)

        voc_complexity_var = voc_complexity(model_voc,word_frequency,word_occur)

        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_node_number" ]= get_node_number(modelA)
        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_voc_length" ]=len(model_voc)
        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_max_path_length" ]= length_key_list[len_length_key_list-1]
        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_min_path_length" ]= length_key_list[0]
        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_intermediate_node" ]= get_node_number(modelA) - len(modelA)
        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_average_word_frequency" ]= get_average_frequency(voc_complexity_var)
        res_csv.loc[ res_csv.query('model_2=="' + model_name + '" & model_2_path_size=='+str(sub_path_A_size) ).index, "model_2_weighted_average_word_frequency" ]= get_weithed_mean_frequency(voc_complexity_var, model_voc_occur)

res_csv.to_csv("validation_metrics_computed_with_metrics.csv")



[nltk_data] Downloading package stopwords to /home/vdang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null
null


In [38]:
file = pd.read_csv("validation_metrics_computed_with_metrics.csv")
AERIS_expe = file[file["model_1"]=="AERIS"]
# AERIS_expe
AERIS_expe[AERIS_expe["tool"]=="COMA"].describe()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,model_1_path_size,model_2_path_size,precision,recall,F1-score,total_matches_in_ground_truth,total_matches,...,model_1_node_number,model_1_average_word_frequency,model_1_weighted_average_word_frequency,model_2_voc_length,model_2_max_path_length,model_2_min_path_length,model_2_intermediate_node,model_2_node_number,model_2_average_word_frequency,model_2_weighted_average_word_frequency
count,28.000000,28.000000,28.000000,28.00000,28.0000,28.000000,28.000000,28.000000,28.0,28.000000,...,28.000000,28.000000,28.000000,28.000000,28.0000,28.0,28.000000,28.000000,28.000000,28.000000
mean,287.928571,287.928571,287.928571,1.50000,3.0000,0.069021,0.294643,0.111645,12.0,51.928571,...,165.250000,0.000143,0.000156,270.285714,4.0000,1.0,206.000000,718.285714,0.000419,0.000257
std,163.633042,163.633042,163.633042,1.13855,2.0367,0.014634,0.048017,0.022157,0.0,5.297998,...,33.923252,0.000004,0.000016,37.824994,2.0367,0.0,105.163013,280.447111,0.000056,0.000101
min,51.000000,51.000000,51.000000,0.00000,0.0000,0.050847,0.250000,0.084507,12.0,41.000000,...,110.000000,0.000140,0.000141,181.000000,1.0000,1.0,0.000000,160.000000,0.000392,0.000186
25%,157.500000,157.500000,157.500000,0.75000,1.0000,0.055556,0.250000,0.090909,12.0,47.000000,...,153.500000,0.000140,0.000142,268.000000,2.0000,1.0,99.000000,452.000000,0.000392,0.000186
50%,276.000000,276.000000,276.000000,1.50000,3.0000,0.070175,0.291667,0.111894,12.0,54.000000,...,179.500000,0.000140,0.000152,289.000000,4.0000,1.0,271.000000,889.000000,0.000392,0.000200
75%,428.500000,428.500000,428.500000,2.25000,5.0000,0.081277,0.333333,0.130672,12.0,55.500000,...,191.250000,0.000143,0.000166,289.000000,6.0000,1.0,275.000000,913.000000,0.000418,0.000328
max,621.000000,621.000000,621.000000,3.00000,6.0000,0.102041,0.416667,0.163934,12.0,59.000000,...,192.000000,0.000150,0.000179,289.000000,7.0000,1.0,283.000000,919.000000,0.000551,0.000471


In [39]:

AERIS_expe[AERIS_expe["tool"]=="euclidian"].describe()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,model_1_path_size,model_2_path_size,precision,recall,F1-score,total_matches_in_ground_truth,total_matches,...,model_1_node_number,model_1_average_word_frequency,model_1_weighted_average_word_frequency,model_2_voc_length,model_2_max_path_length,model_2_min_path_length,model_2_intermediate_node,model_2_node_number,model_2_average_word_frequency,model_2_weighted_average_word_frequency
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.0,27.000000,...,27.000000,27.000000,27.000000,27.000000,27.000000,27.0,27.000000,27.000000,27.000000,27.000000
mean,322.000000,322.000000,322.000000,1.518519,3.111111,0.022699,0.240741,0.041414,12.0,140.740741,...,165.148148,0.000143,0.000156,273.592593,4.111111,1.0,213.629630,738.962963,0.000414,0.000249
std,186.617874,186.617874,186.617874,1.155934,1.987138,0.017009,0.152145,0.030593,0.0,19.488838,...,34.565104,0.000004,0.000016,34.172930,1.987138,0.0,98.954593,263.142179,0.000050,0.000094
min,27.000000,27.000000,27.000000,0.000000,0.000000,0.006410,0.083333,0.011905,12.0,110.000000,...,110.000000,0.000140,0.000141,181.000000,1.000000,1.0,0.000000,160.000000,0.000392,0.000186
25%,184.000000,184.000000,184.000000,0.500000,1.500000,0.006410,0.083333,0.011905,12.0,125.500000,...,139.000000,0.000140,0.000142,277.500000,2.500000,1.0,169.000000,619.000000,0.000392,0.000186
50%,261.000000,261.000000,261.000000,2.000000,3.000000,0.019231,0.250000,0.035714,12.0,156.000000,...,191.000000,0.000140,0.000142,289.000000,4.000000,1.0,271.000000,889.000000,0.000392,0.000200
75%,495.500000,495.500000,495.500000,2.500000,5.000000,0.036912,0.375000,0.067127,12.0,156.000000,...,191.500000,0.000146,0.000171,289.000000,6.000000,1.0,275.000000,913.000000,0.000406,0.000283
max,585.000000,585.000000,585.000000,3.000000,6.000000,0.054545,0.500000,0.098361,12.0,156.000000,...,192.000000,0.000150,0.000179,289.000000,7.000000,1.0,283.000000,919.000000,0.000551,0.000471


In [40]:

AERIS_expe[AERIS_expe["tool"]=="multi"].describe()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,model_1_path_size,model_2_path_size,precision,recall,F1-score,total_matches_in_ground_truth,total_matches,...,model_1_node_number,model_1_average_word_frequency,model_1_weighted_average_word_frequency,model_2_voc_length,model_2_max_path_length,model_2_min_path_length,model_2_intermediate_node,model_2_node_number,model_2_average_word_frequency,model_2_weighted_average_word_frequency
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.0,27.000000,...,27.000000,27.000000,27.000000,27.000000,27.000000,27.0,27.000000,27.000000,27.000000,27.000000
mean,276.851852,276.851852,276.851852,1.518519,3.111111,0.034980,0.391975,0.064109,12.0,140.740741,...,165.148148,0.000143,0.000156,273.592593,4.111111,1.0,213.629630,738.962963,0.000414,0.000249
std,151.637651,151.637651,151.637651,1.155934,1.987138,0.012079,0.076008,0.021250,0.0,19.488838,...,34.565104,0.000004,0.000016,34.172930,1.987138,0.0,98.954593,263.142179,0.000050,0.000094
min,92.000000,92.000000,92.000000,0.000000,0.000000,0.019231,0.250000,0.035714,12.0,110.000000,...,110.000000,0.000140,0.000141,181.000000,1.000000,1.0,0.000000,160.000000,0.000392,0.000186
25%,155.000000,155.000000,155.000000,0.500000,1.500000,0.025641,0.333333,0.047619,12.0,125.500000,...,139.000000,0.000140,0.000142,277.500000,2.500000,1.0,169.000000,619.000000,0.000392,0.000186
50%,266.000000,266.000000,266.000000,2.000000,3.000000,0.032051,0.416667,0.059524,12.0,156.000000,...,191.000000,0.000140,0.000142,289.000000,4.000000,1.0,271.000000,889.000000,0.000392,0.000200
75%,333.500000,333.500000,333.500000,2.500000,5.000000,0.040458,0.416667,0.073663,12.0,156.000000,...,191.500000,0.000146,0.000171,289.000000,6.000000,1.0,275.000000,913.000000,0.000406,0.000283
max,569.000000,569.000000,569.000000,3.000000,6.000000,0.054545,0.500000,0.098361,12.0,156.000000,...,192.000000,0.000150,0.000179,289.000000,7.000000,1.0,283.000000,919.000000,0.000551,0.000471
